# Differential Privacy for PRETSA

In [1]:
!git clone https://github.com/cmalvr/PRETSA

Cloning into 'PRETSA'...
remote: Enumerating objects: 429, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 429 (delta 4), reused 1 (delta 1), pack-reused 424 (from 1)
Receiving objects: 100% (429/429), 11.48 MiB | 36.87 MiB/s, done.
Resolving deltas: 100% (263/263), done.


In [2]:
!pip install -r /content/PRETSA/requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00
  Created wheel for anytree: filename=anytree-2.4.3-py2.py3-none-any.whl size=26707 sha256=bcffc78a7b9b58b13db3663ac6476fe5c8029816ace6ac2bdba4a37839a70018
  Stored in directory: /root/.cache/pip/wheels/f3/d3/8f/9eea88a7fe27ca18e7ebfc23e1012514c8b13a4552aa43be12
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=f3fde297e5ebf3751bc2912830b3ed373ed59f05d442f9c19c761ace56025efe
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built anytree intervaltree


In [3]:
!python /content/PRETSA/dataset_collection.py

/usr/local/lib/python3.11/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 314, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))
Downloaded: /content/PRETSA/downloads/CoSeLoG.xes.gz
Processing CoSeLoG...
parsing log, completed traces :: 100% 1434/1434 [00:00<00:00, 2722.46it/s]
Save

## Running Annotations for three datasets

In [4]:
# Generates all the original annotation duration for each of the three datasets: CoSeLog, Sepsis and Road Traffc
# Generates /content/PRETSA/original_annotation/
!python /content/PRETSA/gc_add_annotations_duration.py with

Processing: CoSeLoG
Writting: /content/PRETSA/original_annotation/CoSeLoG_duration.csv
Reading: /content/PRETSA/csv_outputs/CoSeLoG_dataset.csv
Processing: Sepsis
Writting: /content/PRETSA/original_annotation/Sepsis_duration.csv
Reading: /content/PRETSA/csv_outputs/Sepsis_dataset.csv


## Run Pretsa: Generates Pretsa Log


In [5]:
!python /content/PRETSA/startExperimentsForJournalExtension_PRETSA.py
# Generates: /content/PRETSA/pretsalog/ f"{dataset}_t{t}_k{k}_pretsa.csv"
# These contain the new annotations for duration after Running PRETSA method

 Load Event Log: /content/PRETSA/original_annotation/CoSeLoG_duration.csv
Generated Distance Matrix
 Modified 99 cases for k=4, t=1.0
 Private event log saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k4_pretsa.csv
 Experiment metadata saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k4_pretsa.pickle
 Load Event Log: /content/PRETSA/original_annotation/CoSeLoG_duration.csv
Generated Distance Matrix
 Modified 112 cases for k=8, t=1.0
 Private event log saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k8_pretsa.csv
 Experiment metadata saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k8_pretsa.pickle
 Load Event Log: /content/PRETSA/original_annotation/CoSeLoG_duration.csv
Generated Distance Matrix
 Modified 169 cases for k=16, t=1.0
 Private event log saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k16_pretsa.csv
 Experiment metadata saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k16_pretsa.pickle
 Load Event Log: /content/PRETSA/original_annotation/Sepsis_duration.csv
Generated Dis

## Run Baseline: Generate Baseline log

In [42]:
%cd /content/PRETSA
!git pull

/content/PRETSA
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 271 bytes | 271.00 KiB/s, done.
From https://github.com/cmalvr/PRETSA
   5672748..0c85d91  master     -> origin/master
Updating 5672748..0c85d91
Fast-forward
 calculateAnnotationError.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [19]:
!python /content/PRETSA/gc_generate_baseline_log.py
# Generates: /content/PRETSA/baselinelogs/ Path(filePath).name.replace(".csv","_pretsa_baseline_k%s_t%s.csv" % (kString,tString))
# These contain the new annotations for duration after Running BASELINE method
# Should generate same content as baselogs zip directory

Events before 8577
Events after 8576
Remaining 0.999883409117407
Events before 8577
Events after 8576
Remaining 0.999883409117407
Events before 8577
Events after 8576
Remaining 0.999883409117407
Events before 15214
Events after 15213
Remaining 0.9999342710661233
Events before 15214
Events after 15213
Remaining 0.9999342710661233
Events before 15214
Events after 15213
Remaining 0.9999342710661233


## Calculate Statistics

In [30]:
!python /content/PRETSA/calculateBaselineEventLogStatistics.py #Generates: "baseline_event_logs_statistics/"
!python /content/PRETSA/calculatePRETSAEventLogStatistics.py  #Generates: "PRETSA_event_logs_statistics/"

{'Dataset': 'CoSeLoG', 'k': 4, 'method': 'pretsa', 'variants': 32, 'cases': 1434}
{'Dataset': 'CoSeLoG', 'k': 8, 'method': 'pretsa', 'variants': 27, 'cases': 1434}
{'Dataset': 'CoSeLoG', 'k': 16, 'method': 'pretsa', 'variants': 21, 'cases': 1434}
{'Dataset': 'Sepsis', 'k': 4, 'method': 'pretsa', 'variants': 216, 'cases': 1049}
{'Dataset': 'Sepsis', 'k': 8, 'method': 'pretsa', 'variants': 116, 'cases': 1049}
{'Dataset': 'Sepsis', 'k': 16, 'method': 'pretsa', 'variants': 60, 'cases': 1049}


## Generate Annotations

In [40]:
!python /content/PRETSA/calculateAnnotationsEventLog_baseline.py  #Needs: filePath = dictPath + dataset + "_duration_pretsa_baseline_k" + str(k) + "_t" + str(t) + ".csv" ,Generates: dictPath + "pretsa_baseline_statistics_annotations.csv"
!python /content/PRETSA/calculateAnnotationsEventLog_pretsa.py #Needs:dictPath + dataset + "_duration_t" + tString[t] + "_k" + str(k) + "_pretsa.csv" , generates dictPath + "pretsa_statistics_annotations.csv"
!python /content/PRETSA/generateAnnotationOriginalDataset.py #generates original_annotations_pretsa.csv (located in <dictName>)

## Calculate Annotation Errors

 Relative error of the annotations for each activity

In [43]:
!python /content/PRETSA/calculateAnnotationError.py  #Needs: original_annotations_pretsa.csv, pretsa_baseline_statistics_annotations.csv, pretsa_statistics_annotations.csv